In [101]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
lem = WordNetLemmatizer()

### Data Source

In [102]:
Data = pd.read_excel("cik_list.xlsx")
for i in Data['SECFNAME']:
    Data['SECFNAME'] = Data['SECFNAME'].replace(i,"https://www.sec.gov/Archives/"+i)
pd.set_option('display.max_colwidth', -1)
Data = Data.iloc[:10,:]
Data.head()

C:\Users\Vaibhav Banga\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-001203.txt


### Web Scrapping

In [103]:
Word_count = list()
Total_words = list()
Sentence_Length = list()
for i in Data['SECFNAME']:
    url = i
    r = requests.get(url)
    hc = r.content
    soup = BeautifulSoup(hc,'lxml')
    for t in soup.find_all('table'):
        t.extract()
    text = ""
    for p in soup.find_all('p'):
        text+= p.text
    sent = nltk.sent_tokenize(text)
    text = re.sub("[^a-zA-Z]"," ",text)
    text = re.sub("\s+"," ",text)
    text = text.lower()
    text = nltk.word_tokenize(text)
    for i in text:
        if(len(i)==1):
            text.remove(i)
    text = [lem.lemmatize(word) for word in text if word not in set(stopwords.words("english"))]
    Total_words.append(text)
    Word_count.append(len(text))
    Sentence_Length.append(len(sent))
print(Word_count)
print(len(Total_words))
print(Sentence_Length)

[85739, 268, 546, 41729, 659, 8103, 13787, 8550, 268, 40570]
10
[3860, 14, 20, 2461, 25, 460, 733, 489, 14, 2416]


### Positive_Score || Negative_Score  ||        Polarity_Score

In [104]:
positive_word = list()
negative_word = list()
negative_score = list()
positive_score = list()
Polarity = list()
for i in Total_words:
    for j in i:
        score = SentimentIntensityAnalyzer().polarity_scores(j)
        if(score['neg']==1.0):
            negative_word.append(j)
        elif(score['pos']==1.0):
            positive_word.append(j)
    negative_score.append(len(negative_word))
    positive_score.append(len(positive_word))
    polarity = (len(positive_word)-len(negative_word))/(len(positive_word)+len(negative_word))+0.000001
    Polarity.append(polarity)
    negative_word.clear()
    positive_word.clear()

### Positive_word_Proportion || Negative_word_proportion

In [105]:
Negative_word_proportion = list()
for i in range(len(Total_words)):
    p = (negative_score[i]/Word_count[i])*100
    Negative_word_proportion.append(p)
Positive_word_proportion = list()
for i in range(len(Total_words)):
    p = (positive_score[i]/Word_count[i])*100
    Positive_word_proportion.append(p)

### Average_sentence_count

In [106]:
avg = list()
for i in range(len(Total_words)):
    a = Word_count[i]/Sentence_Length[i]
    avg.append(a)
print(avg)

[22.212176165803108, 19.142857142857142, 27.3, 16.956115400243803, 26.36, 17.61521739130435, 18.80900409276944, 17.484662576687118, 19.142857142857142, 16.792218543046356]


### Complex_word

In [107]:
Percentage_of_Complex = list()
Fog_index = list()
b = 'aeiou'
c = 0
d = list()
cwd = list()
for i in Total_words:
    for j in i:
        if(len(j)>=2)  and  ((j[-2]+j[-1]!='es') and (j[-2]+j[-1]!='ed')):
            for k in j:
                if (k.lower() in b):
                    c = c+1
                    j[1:]
            if(c>2):
                d.append(j)
    cwd.append(len(d))
    d.clear()   
for j in range(len(cwd)):
    Percentage_of_Complex.append((cwd[j]/len(Total_words[j]))*100)
for k in range(len(cwd)):
    Fog_index.append(0.4*(avg[k]+Percentage_of_Complex[j]))
print(Percentage_of_Complex)
print(Fog_index)

[90.96560491724885, 91.7910447761194, 88.27838827838828, 89.81283999137291, 88.01213960546282, 88.18955942243613, 87.95241894538334, 87.47368421052632, 92.16417910447761, 89.78802070495439]
[44.800078748303, 43.57235113912461, 46.83520828198176, 42.69765444207928, 46.45920828198176, 42.9612952385035, 43.438809919089536, 42.9090733126566, 43.57235113912461, 42.632095699200306]


In [108]:
constr_dict = pd.read_excel("constraining_dictionary.xlsx")
unconstr_dict = pd.read_excel("uncertainty_dictionary.xlsx")

### Constraining

In [109]:
constraining = list()
for i in constr_dict['Word']:
    constraining.append(i.lower())
print(constraining)

['abide', 'abiding', 'bound', 'bounded', 'commit', 'commitment', 'commitments', 'commits', 'committed', 'committing', 'compel', 'compelled', 'compelling', 'compels', 'comply', 'compulsion', 'compulsory', 'confine', 'confined', 'confinement', 'confines', 'confining', 'constrain', 'constrained', 'constraining', 'constrains', 'constraint', 'constraints', 'covenant', 'covenanted', 'covenanting', 'covenants', 'depend', 'dependance', 'dependances', 'dependant', 'dependencies', 'dependent', 'depending', 'depends', 'dictate', 'dictated', 'dictates', 'dictating', 'directive', 'directives', 'earmark', 'earmarked', 'earmarking', 'earmarks', 'encumber', 'encumbered', 'encumbering', 'encumbers', 'encumbrance', 'encumbrances', 'entail', 'entailed', 'entailing', 'entails', 'entrench', 'entrenched', 'escrow', 'escrowed', 'escrows', 'forbade', 'forbid', 'forbidden', 'forbidding', 'forbids', 'impair', 'impaired', 'impairing', 'impairment', 'impairments', 'impairs', 'impose', 'imposed', 'imposes', 'impos

### Unconstraining

In [110]:
uncertainty = list()
for i in unconstr_dict['Word']:
    uncertainty.append(i.lower())
print(uncertainty)

['abeyance', 'abeyances', 'almost', 'alteration', 'alterations', 'ambiguities', 'ambiguity', 'ambiguous', 'anomalies', 'anomalous', 'anomalously', 'anomaly', 'anticipate', 'anticipated', 'anticipates', 'anticipating', 'anticipation', 'anticipations', 'apparent', 'apparently', 'appear', 'appeared', 'appearing', 'appears', 'approximate', 'approximated', 'approximately', 'approximates', 'approximating', 'approximation', 'approximations', 'arbitrarily', 'arbitrariness', 'arbitrary', 'assume', 'assumed', 'assumes', 'assuming', 'assumption', 'assumptions', 'believe', 'believed', 'believes', 'believing', 'cautious', 'cautiously', 'cautiousness', 'clarification', 'clarifications', 'conceivable', 'conceivably', 'conditional', 'conditionally', 'confuses', 'confusing', 'confusingly', 'confusion', 'contingencies', 'contingency', 'contingent', 'contingently', 'contingents', 'could', 'crossroad', 'crossroads', 'depend', 'depended', 'dependence', 'dependencies', 'dependency', 'dependent', 'depending'

### Constraining_Score

In [111]:
s = 0
C_Sc = list()
a = list()
for i in Total_words:
    for j in i:
        for k in constraining:
            if(j == k):
                s = s+1
                if(s>0):
                    a.append(k)
    C_Sc.append(len(a))
    a.clear()

### Constraining_Word_Proportion

In [112]:
constraining_proportion = list()
for i in range(len(C_Sc)):
    p = (C_Sc[i]/len(constraining))*100
    constraining_proportion.append(p)
uncertainty_proportion = list()
for j in range(len(UC_Sc)):
    P = (UC_Sc[j]/len(uncertainty))*100
    uncertainty_proportion.append(P)

### Unconstraining_Score

In [113]:
s = 0
UC_Sc = list()
a = list()
for i in Total_words:
    for j in i:
        for k in uncertainty:
            if(j == k):
                s = s+1
                if(s>0):
                    a.append(k)
    UC_Sc.append(len(a))
    a.clear()

In [114]:
Data['Positive_Score'] = positive_score
Data['Negative_Score'] = negative_score
Data['Polarity_Score'] = Polarity
Data['Average_Sentence_Length'] = avg
Data['Percentage_of_complex_words'] = Percentage_of_Complex
Data['Fog_Index'] = Fog_index
Data['Complex_word_count'] = cwd
Data['Word_Count'] = Word_count
Data['uncertainty_score'] = UC_Sc
Data['constraining_score'] = C_Sc
Data['Positive_word_proportion'] = Positive_word_proportion
Data['Negative_word_proportion'] = Negative_word_proportion
Data['uncertainty_word_proportion'] = uncertainty_proportion
Data['constraining_word_proportion'] = constraining_proportion
Data

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,Positive_Score,Negative_Score,Polarity_Score,Average_Sentence_Length,Percentage_of_complex_words,Fog_Index,Complex_word_count,Word_Count,uncertainty_score,constraining_score,Positive_word_proportion,Negative_word_proportion,uncertainty_word_proportion,constraining_word_proportion
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt,8707,1936,0.636194,22.212176,90.965605,44.800079,77993,85739,924,1462,10.155239,2.258016,311.111111,794.565217
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-001001.txt,24,7,0.548388,19.142857,91.791045,43.572351,246,268,4,5,8.955224,2.611940,284.511785,2.717391
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-000783.txt,21,0,1.000001,27.300000,88.278388,46.835208,482,546,8,5,3.846154,0.000000,2.693603,2.717391
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002145.txt,3715,885,0.615218,16.956115,89.812840,42.697654,37478,41729,534,688,8.902682,2.120827,179.797980,373.913043
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-001203.txt,23,1,0.916668,26.360000,88.012140,46.459208,580,659,10,4,3.490137,0.151745,3.367003,2.173913
5,3662,SUNBEAM CORP/FL/,199811,1998-11-25,10-Q/A,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002278.txt,616,245,0.430895,17.615217,88.189559,42.961295,7146,8103,175,89,7.602123,3.023572,58.922559,48.369565
6,3662,SUNBEAM CORP/FL/,199812,1998-12-22,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002401.txt,1162,477,0.417939,18.809004,87.952419,43.438810,12126,13787,272,267,8.428229,3.459781,91.582492,145.108696
7,3662,SUNBEAM CORP/FL/,199812,1998-12-22,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002402.txt,644,258,0.427939,17.484663,87.473684,42.909073,7479,8550,193,101,7.532164,3.017544,64.983165,54.891304
8,3662,SUNBEAM CORP/FL/,199903,1999-03-31,NT 10-K,https://www.sec.gov/Archives/edgar/data/3662/0000950172-99-000362.txt,24,7,0.548388,19.142857,92.164179,43.572351,247,268,4,5,8.955224,2.611940,2.693603,2.717391
9,3662,SUNBEAM CORP/FL/,199905,1999-05-11,10-K,https://www.sec.gov/Archives/edgar/data/3662/0000950170-99-000775.txt,3639,981,0.575326,16.792219,89.788021,42.632096,36427,40570,611,472,8.969682,2.418043,205.723906,256.521739


In [115]:
Data.to_excel('Result.xlsx')